In [4]:
# check python version
import sys
print(sys.version)

3.10.9 (main, Dec 15 2022, 10:44:50) [Clang 14.0.0 (clang-1400.0.29.202)]


## Data Pipeline

In this project I will showcase how to ETL female literacy rate data from the World Bank by utilizing its API. 

More precisely, this table: *Literacy rate, adult female (% of females ages 15 and above)*

My ultimate goal is to use this data to create a data analysis dashboard that shows the gender disparity in the STEM field, particularly in Latin America. This dataset will be part of a collection of datasets I will use to make my analysis

If you want to know more about this API check the documentation here: https://datahelpdesk.worldbank.org/knowledgebase/articles/898581-api-basic-call-structures

### Step 1: Data Extraction
We will start by extracting data from the UNESCO data portal. For this, we need to use web scraping tools to automate the data extraction process.

In [11]:
# Install dependencies
!pip install -r requirements.txt

In [12]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

##### API endpoint and parameters
I am defining the parameters of my extracted data here in a param dictionary. The `requests` library automatically encodes these parameters into the url. The parameters are available in the [API's documentation](https://datahelpdesk.worldbank.org/knowledgebase/articles/898581-api-basic-call-structures). This improves code readability.

* **JSON**: I want for the raw data to be extracted in json format as it's easier to read, more compact, and therefore faster to parse through 
* **date**: This is the date range of my data

In [124]:
# World Bank API endpoint URL

url = "https://api.worldbank.org/v2/country/all/indicator/SE.ADT.LITR.FE.ZS"

# Establish the parameters you want your data to have
params = {
    "format": "json",
    "date": "2000:2023",
    "per_page": 1000
}

**Important Note**:
We want to ensure we are extracting all the data. Since this is API returns paginated responses we have to ensure we are fetching all the pages

There are multiple things happening within this function. 

We know that the first record of the json response for each page `data[0]` is metadata , *I discovered this when I extracted a single page*. Therefore: 
1. We are discarding the metadata by populating the list `all_data` with the actual data which is inside the second record in the json response `data[1]`
2. Once we don't find any more data records we break out of the while loop and return all the data in a single list!

In [165]:
# Return all the pages from the paginated API responses

def fetch_all(url, params):
    all_data = []
    page = 1
    
    while True:
        params['page'] = page
        response = requests.get(url, params=params)
        data = response.json()
        
        # Check if there are records
        if len(data) > 1 and data[1]:
            all_data.extend(data[1])
            page += 1
        else:
            break
    return all_data

In [166]:
data = fetch_all(url, params)

### Step 2. Data Transformation

##### Convert to a raw data to DataFrame
By checking the first few lines here we can see there is a metadata record at the beginning. We want to remove this.

In [170]:
df = pd.DataFrame(data)

##### Exploratory Analysis

We want to explore how the data looks, clean it up and get it ready for analysis

In [172]:
df.shape

(14098, 8)

In [173]:
df.columns

Index(['indicator', 'country', 'countryiso3code', 'date', 'value', 'unit',
       'obs_status', 'decimal'],
      dtype='object')

In [174]:
df.head()

,indicator,country,countryiso3code,date,value,unit,obs_status,decimal
0,"{'id': 'SE.ADT.LITR.FE.ZS', 'value': 'Literacy...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2023,NaN,,,0
1,"{'id': 'SE.ADT.LITR.FE.ZS', 'value': 'Literacy...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2022,67.545410,,,0
2,"{'id': 'SE.ADT.LITR.FE.ZS', 'value': 'Literacy...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2021,67.454010,,,0
3,"{'id': 'SE.ADT.LITR.FE.ZS', 'value': 'Literacy...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2020,67.536987,,,0
4,"{'id': 'SE.ADT.LITR.FE.ZS', 'value': 'Literacy...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2019,67.289780,,,0


First I am exploring the fields of the *indicator* column

In [187]:
df['indicator'][0]

{'id': 'SE.ADT.LITR.FE.ZS',
 'value': 'Literacy rate, adult female (% of females ages 15 and above)'}

In [188]:
df['indicator'][0]['id']

'SE.ADT.LITR.FE.ZS'

In [182]:
df['indicator'][12332]

{'id': 'SE.ADT.LITR.FE.ZS',
 'value': 'Literacy rate, adult female (% of females ages 15 and above)'}

**Process notes**: They might all be the same? I will extract the ids and values into a separate column each

In [190]:
# Place the indicator id in a separate column for detailed examination

df['indicator_id'] = df['indicator'].apply(lambda x: x['id'])

In [194]:
df['indicator_id'].unique() # It seems like this is not of much help - it's just the data code

array(['SE.ADT.LITR.FE.ZS'], dtype=object)

In [195]:
# Place the indicator value in a separate column for detailed examination

df['indicator_val'] = df['indicator'].apply(lambda x: x['value'])

In [196]:
df['indicator_val'].unique() # same for this - not so relevant

array(['Literacy rate, adult female (% of females ages 15 and above)'],
      dtype=object)

**Process notes**: I decided to drop the indicator column and its derived columns as they're not relevant for the purposes of extracting this data

In [197]:
df.drop(columns=['indicator', 'indicator_id', 'indicator_val'], inplace=True)

In [198]:
df.columns

Index(['country', 'countryiso3code', 'date', 'value', 'unit', 'obs_status',
       'decimal'],
      dtype='object')

**Process notes**: Now, I'll explore the country column

In [200]:
df['country'][0]

{'id': 'ZH', 'value': 'Africa Eastern and Southern'}

In [201]:
df['country_id'] = df['country'].apply(lambda x: x['id'])

In [202]:
df['country_val'] = df['country'].apply(lambda x: x['value'])

In [203]:
df['country_id'].unique()

array(['ZH', 'ZI', '1A', 'S3', 'B8', 'V2', 'Z4', '4E', 'T4', 'XC', 'Z7',
       '7E', 'T7', 'EU', 'F1', 'XE', 'XD', 'XF', 'ZT', 'XH', 'XI', 'XG',
       'V3', 'ZJ', 'XJ', 'T2', 'XL', 'XO', 'XM', 'XN', 'ZQ', 'XQ', 'T3',
       'XP', 'XU', 'XY', 'OE', 'S4', 'S2', 'V4', 'V1', 'S1', '8S', 'T5',
       'ZG', 'ZF', 'T6', 'XT', '1W', 'AF', 'AL', 'DZ', 'AS', 'AD', 'AO',
       'AG', 'AR', 'AM', 'AW', 'AU', 'AT', 'AZ', 'BS', 'BH', 'BD', 'BB',
       'BY', 'BE', 'BZ', 'BJ', 'BM', 'BT', 'BO', 'BA', 'BW', 'BR', 'VG',
       'BN', 'BG', 'BF', 'BI', 'CV', 'KH', 'CM', 'CA', 'KY', 'CF', 'TD',
       'JG', 'CL', 'CN', 'CO', 'KM', 'CD', 'CG', 'CR', 'CI', 'HR', 'CU',
       'CW', 'CY', 'CZ', 'DK', 'DJ', 'DM', 'DO', 'EC', 'EG', 'SV', 'GQ',
       'ER', 'EE', 'SZ', 'ET', 'FO', 'FJ', 'FI', 'FR', 'PF', 'GA', 'GM',
       'GE', 'DE', 'GH', 'GI', 'GR', 'GL', 'GD', 'GU', 'GT', 'GN', 'GW',
       'GY', 'HT', 'HN', 'HK', 'HU', 'IS', 'IN', 'ID', 'IR', 'IQ', 'IE',
       'IM', 'IL', 'IT', 'JM', 'JP', 'JO', 'KZ', 'K

In [204]:
df['country_val'].unique()

array(['Africa Eastern and Southern', 'Africa Western and Central',
       'Arab World', 'Caribbean small states',
       'Central Europe and the Baltics', 'Early-demographic dividend',
       'East Asia & Pacific',
       'East Asia & Pacific (excluding high income)',
       'East Asia & Pacific (IDA & IBRD countries)', 'Euro area',
       'Europe & Central Asia',
       'Europe & Central Asia (excluding high income)',
       'Europe & Central Asia (IDA & IBRD countries)', 'European Union',
       'Fragile and conflict affected situations',
       'Heavily indebted poor countries (HIPC)', 'High income',
       'IBRD only', 'IDA & IBRD total', 'IDA blend', 'IDA only',
       'IDA total', 'Late-demographic dividend',
       'Latin America & Caribbean',
       'Latin America & Caribbean (excluding high income)',
       'Latin America & the Caribbean (IDA & IBRD countries)',
       'Least developed countries: UN classification',
       'Low & middle income', 'Low income', 'Lower middle in

**Process notes**: This data is very relevant! I will clean it up more as I am focusing my analysis on the LATAM region